In [34]:
import re
import numpy as np
from random import shuffle
import pandas as pd

In [35]:
# Preprocess the Documents
def preprocess_document(doc):
    doc = doc.lower()
    doc = re.sub(r'\W+', ' ', doc)
    return doc

In [36]:
# Generate Shingles
def shingle(text, k):
    shingle_set = set()
    for i in range(len(text) - k + 1):
        shingle_set.add(text[i:i+k])
    return shingle_set

In [37]:
# Sample documents
#a = "flying fish flew by the space station"
#b = "he will not allow you to bring your sticks of dynamite and pet armadillo along"
#c = "he figured a few sticks of dynamite were easier than a fishing pole to catch an armadillo"

a= "The quick brown fox jumps over the lazy dog."
#b="Never jump over the lazy dog quickly."
c="A fox is quick and brown and it jumps over dogs."
b="The quick brown fox jumps over the lazy dog1."

In [38]:

# Preprocess and generate shingles for each document
k = 2
a_shingles = shingle(preprocess_document(a), k)
b_shingles = shingle(preprocess_document(b), k)
c_shingles = shingle(preprocess_document(c), k)

print("Shingles for document a:", a_shingles)
print("Shingles for document b:", b_shingles)
print("Shingles for document c:", c_shingles)

Shingles for document a: {' l', 'ui', 'br', 'ox', 'he', 'r ', ' b', ' o', 'g ', ' d', 'wn', 'e ', 'er', 'la', 've', 'az', ' f', 'ck', 'th', 'do', ' q', 'n ', 'y ', 'mp', 'k ', 'fo', ' t', 'ro', 'ov', 'um', 'ps', 'ic', 's ', ' j', 'ju', 'zy', 'ow', 'x ', 'og', 'qu'}
Shingles for document b: {' l', 'ui', 'br', 'ox', 'he', 'r ', '1 ', ' b', 'g1', ' o', ' d', 'wn', 'e ', 'er', 'la', 've', 'az', ' f', 'ck', 'th', 'do', ' q', 'n ', 'y ', 'mp', 'k ', 'fo', ' t', 'ro', 'ov', 'um', 'ps', 'ic', 's ', ' j', 'ju', 'zy', 'ow', 'x ', 'og', 'qu'}
Shingles for document c: {'ui', 'is', 'gs', 'br', 'ox', 'r ', ' b', ' a', ' o', ' d', 'wn', 'er', 've', ' f', 't ', 'a ', 'an', 'd ', 'ck', 'do', ' q', 'n ', 'it', 'nd', 'mp', 'k ', 'fo', ' i', 'ro', 'ov', 'um', 'ps', 'ic', 's ', ' j', 'ju', 'ow', 'x ', 'og', 'qu'}


In [39]:
# Create vocabulary
vocab = list(a_shingles.union(b_shingles).union(c_shingles))
print("Vocabulary:", vocab)

Vocabulary: ['ui', 'is', 'r ', 'he', '1 ', ' b', ' a', ' d', 'wn', 'la', 't ', 'a ', 'ck', ' q', 'n ', 'it', 'mp', 'nd', 'k ', 'ov', ' j', 'ju', 'zy', 'ow', 'x ', 'qu', ' l', 'gs', 'br', 'ox', 'g1', ' o', 'g ', 'er', 'e ', 've', 'az', ' f', 'an', 'd ', 'th', 'do', 'y ', 'fo', ' t', ' i', 'ro', 'um', 'ps', 'ic', 's ', 'og']


In [40]:
# One-Hot Encoding
def one_hot_encode(shingles, vocab):
    return [1 if shingle in shingles else 0 for shingle in vocab]

a_1hot = one_hot_encode(a_shingles, vocab)
b_1hot = one_hot_encode(b_shingles, vocab)
c_1hot = one_hot_encode(c_shingles, vocab)

print("One-Hot Encoding for document a:", a_1hot)
print("One-Hot Encoding for document b:", b_1hot)
print("One-Hot Encoding for document c:", c_1hot)

One-Hot Encoding for document a: [1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
One-Hot Encoding for document b: [1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
One-Hot Encoding for document c: [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1]


In [41]:
# Create hash functions
def create_hash_func(size):
    hash_ex = list(range(1, size + 1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size, num_perm):
    hashes = []
    for _ in range(num_perm):
        hashes.append(create_hash_func(vocab_size))
    return hashes

In [42]:
# Generate MinHash signatures
num_perm = 128
minhash_funcs = build_minhash_func(len(vocab), num_perm)

def create_signature(one_hot_vector, minhash_funcs):
    signature = []
    for func in minhash_funcs:
        for i in range(1, len(vocab) + 1):
            idx = func.index(i)
            if one_hot_vector[idx] == 1:
                signature.append(idx)
                break
    return signature

a_signature = create_signature(a_1hot, minhash_funcs)
b_signature = create_signature(b_1hot, minhash_funcs)
c_signature = create_signature(c_1hot, minhash_funcs)

print("MinHash Signature for document a:", a_signature[:10])
print("MinHash Signature for document b:", b_signature[:10])
print("MinHash Signature for document c:", c_signature[:10])

MinHash Signature for document a: [51, 19, 44, 13, 43, 34, 41, 44, 42, 48]
MinHash Signature for document b: [51, 19, 30, 13, 43, 34, 41, 44, 42, 48]
MinHash Signature for document c: [51, 19, 38, 27, 43, 35, 15, 45, 18, 48]


In [43]:
class MinHashLSH:
    def __init__(self, threshold=0.5, num_perm=128, bands=10):
        self.threshold = threshold
        self.num_perm = num_perm
        self.bands = bands
        self.rows = num_perm // bands
        self.buckets = [{} for _ in range(bands)]
    
    def _get_band_hashes(self, minhash):
        band_hashes = []
        for i in range(self.bands):
            band = tuple(minhash[i*self.rows:(i+1)*self.rows])
            band_hashes.append(hash(band))
        return band_hashes
    
    def insert(self, key, minhash):
        band_hashes = self._get_band_hashes(minhash)
        for i, band_hash in enumerate(band_hashes):
            if band_hash not in self.buckets[i]:
                self.buckets[i][band_hash] = []
            self.buckets[i][band_hash].append(key)
    
    def query(self, minhash):
        band_hashes = self._get_band_hashes(minhash)
        candidates = set()
        for i, band_hash in enumerate(band_hashes):
            if band_hash in self.buckets[i]:
                candidates.update(self.buckets[i][band_hash])
        return list(candidates)

# Apply LSH
lsh = MinHashLSH(threshold=0.5, num_perm=num_perm, bands=10)
lsh.insert('a', a_signature)
lsh.insert('b', b_signature)
lsh.insert('c', c_signature)

In [44]:

# Query for similar documents to document 'a'
similar_docs = lsh.query(a_signature)
print("Documents similar to document 'a':", similar_docs)

Documents similar to document 'a': ['b', 'a']
